In [1]:
import pandas as pd
import os, glob

from owlready2 import *
import owlready2
print(owlready2.VERSION)


import importlib.util
import sys
spec = importlib.util.spec_from_file_location("rdfutils", "../../../utils/rdfutils.py")
u = importlib.util.module_from_spec(spec)
sys.modules["rdfutils"] = u
spec.loader.exec_module(u)

from datetime import datetime

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

%load_ext autoreload
%autoreload 2


0.40


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
from dotenv import load_dotenv
load_dotenv('.env')
import json, requests
import importlib.util
import sys

spec = importlib.util.spec_from_file_location("llm", "../../../utils/llm.py")
h = importlib.util.module_from_spec(spec)
sys.modules["llm"] = h
spec.loader.exec_module(h)

URL = os.getenv("KG_URL_FCT")
TOK3N = os.getenv("KG_TOKEN")

In [3]:
onto = get_ontology("WIP.owl").load()
dIDct = u.createDict(onto) 

PBNThing
BenefReturn
Benef
Article
Risk
ISO_Scale
RiskHealth
RiskType
Stakeholder
Stakeholder_Type
Technology
ISO_Impact
ISO_Purpose
StakeholderGroup
StakeholderSubgroup
TechGroup
TechSubgroup
Mitigation
BP_Enabler
BP_Transmission
Blueprint
BP_Scale
BP_Phase
BP_Permanent
BP_Type
BP_Intervention
PBNCategory
RiskGroup
RiskSubgroup
CAO_Group
CAO_Subgroup
BP_Theme
BP_Category
aBlueprint


In [6]:
[str(x) for x in onto.classes() if "Sub" in str(x)]

['WIP.StakeholderSubgroup',
 'WIP.TechSubgroup',
 'WIP.RiskSubgroup',
 'WIP.CAO_Subgroup']

In [7]:
comments = u.checkComments(onto)

ID: 0 	 Author: Luc Jonveaux
ID: 1 	 License: CC BY-NC-SA
ID: 2 	 Language: English
ID: 3 	 Project: PROBONO
ID: 4 	 Task: T3.5
ID: 5 	 Description: Creation of a knowledge graph based on a litterature review, augmented by use of LLMs.
ID: 6 	 Changes from: 0.0
ID: 7 	 Changes: Creation of the knowledge graph
ID: 8 	 Changes from: 0.1
ID: 9 	 Changes: Adding relations, simplified ontology, closest items, and descriptions
ID: 10 	 Repository: https://github.com/mm80843/T3.5/
ID: 11 	 TODOs: Quite a number! Creating metaRisks, making it accessible..
ID: 12 	 Creation: 11/11/2023
ID: 13 	 Version: 0.3
ID: 14 	 VersionComment: New blueprints added in 0.3
ID: 15 	 Library: owlready2==0.45


# Testing custom SPARQL queries

In [8]:
from IPython.display import display, Markdown

In [9]:
def EX(REQ):
    return list(default_world.sparql(REQ))
    
def ask(QUESTION,prefix="sparql",overwrite=False,MODEL="gpt-3.5-turbo-1106",seed=""):
    CONTEXT  = "You are an expert in the sparql language."
    CONTEXT += "You will have to provide  a sparql request that counts the number of classes in a knowledge graph, \
          please answer with the request as between ```sparql ``` tags, and then provide details of how the request is built."
    REQ = {
        "context": CONTEXT,
        "question": QUESTION,
        "model": MODEL,
        "token": TOK3N,
        "overwrite": overwrite,
        "source": "local-sparqlqueries",
        "seed" : seed
    }

    H = h.hashme(CONTEXT+QUESTION+seed)
    cached = "cache/"+prefix+"_"+H+".json"
    if not os.path.isfile(cached) or overwrite:
        x = requests.post(URL+"ask/", json = REQ)
        #print(x.text)
        answer = json.loads(x.text)["answer"]
        h.svt(cached,answer)
    else:
        answer = h.ldt(cached)
    A = [x.replace("sparql\n","").strip() for x in answer.split("```") if len(x)]
    display(Markdown("### Explanation"))
    display(Markdown((A[-1])))
    display(Markdown("--------\n### Code\n"))
    display(Markdown("```sparql\n"+(A[-2])+"\n```"))
    display(Markdown("--------\n### Request result"))
    B = EX(A[-2])
    return B

In [10]:
specifics ="""The ontology we are reviewing has several classes:
* 'Risk': A specific risk
* 'Mitigation': A mitigation against a risk
* 'Technology': A technology used to mitigate a risk
* 'TechGroup' : A category used to classify Technology
* 'Stakeholder': People related to risks
* 'Article': Sources of information
They are linked using the following properties:
* A 'Risk' has a 'Mitigation' : noted with the property : 'has_RiskMitigation' 
* A 'Risk' has a 'Technology' : noted with the property : 'has_RiskTechnology'
* A 'Risk' impacts a 'Stakeholder' : noted with the property : Risk->'has_RiskSubject'->Stakeholder . Inverse is 'has_SubjectRisk' .
* A 'Risk' can be mitigated by a 'Stakeholder' :  noted with the property : 'has_RiskOwner'
* A 'TechGroup' contains 'Technology', noted with the property:  Technology->'has_TechGroup'->TechGroup . Inverse is 'has_GroupTech' .
Note that all classes have capital letters as part of their names.

This ontology can be added in the sparql request introducing:
PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>
"""

In [9]:
A = ask(specifics+"How do I count the number of classes",overwrite=False,MODEL="gpt-3.5-turbo-1106",seed="")
A

### Explanation

In this SPARQL request, we use the `PREFIX` keyword to define the namespace prefix `pbn` for the ontology. The ontology specifics its full URL using this prefix. 

The `SELECT` clause is used to specify the variables we want to retrieve from the query results. In this case, we want to count the total number of distinct classes, so we use the `COUNT` function to count the number of distinct values of `?class`, and alias it as `?classCount`.

The `WHERE` clause is used to specify the graph patterns that data must match. In this case, we are looking for any subject (`?s`) that is a member of some class (`?class`) using the `a` property, which is used in RDF to denote the class membership.

--------
### Code


```sparql
PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT (COUNT(DISTINCT ?class) as ?classCount)
WHERE {
  ?s a ?class .
}
```

--------
### Request result

[[38]]

In [10]:
A = ask(specifics+"How do I count how many Technology belong to each TechGroup, and sort in ascending order (largest TechGroup first), keeping the top 7?",overwrite=False,MODEL="gpt-3.5-turbo-1106",seed="3") # gpt-4-0613
A

### Explanation

In the given SPARQL request, we first specify the prefix `pbn` to abbreviate the full ontology URI. Next, we use a `SELECT` statement to retrieve the count of technologies that belong to each TechGroup, along with the TechGroup itself. We use the `COUNT` aggregate function to count the number of instances of `?technology` for each `?techGroup`. The `WHERE` clause specifies that `?technology` must be an instance of the `pbn:Technology` class and that it must have a `pbn:has_TechGroup` relationship with `?techGroup`. We then group the results by `?techGroup` using the `GROUP BY` clause. The `ORDER BY` clause sorts the results in descending order of the technology count, and the `LIMIT` clause ensures that only the top 7 results are returned.

--------
### Code


```sparql
PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT (COUNT(?technology) AS ?count)
       ?techGroup
WHERE {
  ?technology a pbn:Technology .
  ?technology pbn:has_TechGroup ?techGroup .
}
GROUP BY ?techGroup
ORDER BY DESC(?count)
LIMIT 7
```

--------
### Request result

[[3890, WIP.PBN__TechGroup_0],
 [197, WIP.PBN__Technology_2164],
 [176, WIP.PBN__Technology_2100],
 [136, WIP.PBN__Technology_1316],
 [128, WIP.PBN__Technology_2226],
 [122, WIP.PBN__Technology_2270],
 [108, WIP.PBN__Technology_2274]]

In [11]:
EX("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>
SELECT ?TechGroup (COUNT(?technology) AS ?techCount)
WHERE {
  ?technology a pbn:Technology ;
             pbn:has_TechGroup ?TechGroupObj .
  ?TechGroupObj a pbn:TechGroup ;
               rdfs:label ?TechGroup .
} 
GROUP BY ?TechGroup
ORDER BY DESC(?techCount)
LIMIT 7""")

[['Water control', 3890],
 ['Waste management', 3890],
 ['Urban planning', 3890],
 ['Supply-chain', 3890],
 ['Sensors and monitoring', 3890],
 ['Privacy enhancing technologies', 3890],
 ['Infrastructure management', 3890]]

In [43]:
A = ask(specifics+"How do I identify the TechGroup, and its label, that has the most Technology items linked to a Risk?",overwrite=False,MODEL="gpt-3.5-turbo-1106",seed="5")
A

### Explanation

PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT ?techGroup (COUNT(?technology) AS ?techCount)
WHERE {
  ?risk a pbn:Risk ;
        pbn:has_RiskTechnology ?technology .
  ?technology pbn:has_TechGroup ?techGroup .
}
GROUP BY ?techGroup
ORDER BY DESC(?techCount)
LIMIT 1

--------
### Code


IndexError: list index out of range

In [44]:
A = ask(specifics+"Let's take the 'pbn:PBN__Risk_1', one of the Risk instance. How do I identify the different individual TechGroup to which the Technology instances linked to the Risk (through has_RiskTechnology) ?",overwrite=False,MODEL="gpt-3.5-turbo-1106",seed="5")
A

### Explanation

In the given scenario, to identify the different individual 'TechGroup' to which the 'Technology' instances are linked to the 'Risk' through the 'has_RiskTechnology' property, the first step is to count the number of classes in the knowledge graph using a SPARQL query. The query uses the 'PREFIX' clause to establish a namespace for the ontology. It then selects and counts all distinct classes using the 'SELECT' and 'COUNT' clauses respectively. The 'WHERE' clause is used to find all classes that are defined in the ontology (i.e., their type is owl:Class) and have a URI starting with the provided namespace. The filter condition ensures that only classes from the specified ontology are counted.

--------
### Code


```sparql
PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT (COUNT(DISTINCT ?class) as ?classCount)
WHERE {
    ?class a owl:Class .
    FILTER(STRSTARTS(STR(?class), STR(pbn:)))
}
```

--------
### Request result

ParsingError: ("Error at PNAME_NS:'pbn:'", 188)

In [45]:
A = ask(specifics+"How do I count the top 5 classes that have the children with most instances?",overwrite=False,MODEL="gpt-3.5-turbo-1106",seed="")
A

### Explanation

In the given ontology, we want to count the top 5 classes that have the most instances. To achieve this, we begin by using the `PREFIX` keyword to declare the namespace associated with the ontology. We then use the `SELECT` clause to retrieve the classes and count their instances. In the `WHERE` clause, we match each instance (`?instance`) with its corresponding class (`?class`) using the `a` predicate denoting a type relationship. We then group the results by the class and count the instances for each class using the `COUNT` function, before ordering the results in descending order based on the instance count. Lastly, we use the `LIMIT 5` clause to only retrieve the top 5 classes with the most instances.

--------
### Code


```sparql
PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT ?class (COUNT(?instance) AS ?instanceCount)
WHERE {
  ?instance a ?class .
}
GROUP BY ?class
ORDER BY DESC(?instanceCount)
LIMIT 5
```

--------
### Request result

[[owl.NamedIndividual, 88171],
 [WIP.Technology, 23139],
 [WIP.Mitigation, 22950],
 [WIP.Risk, 21159],
 [WIP.Stakeholder, 16128]]

In [46]:
A = ask(specifics+"I have a specific Risk. How can I search for the list of Stakeholder impacted?",overwrite=False,MODEL="gpt-3.5-turbo-1106",seed="")

### Explanation

This SPARQL query is built to count the number of classes in the ontology. It uses the `PREFIX` statement to define the namespace `pbn` as shorthand for the ontology URI. The `SELECT` clause is used to select and return the count of distinct classes. The `COUNT` and `DISTINCT` are used to ensure that each class is counted only once, even if it appears multiple times in the data. The `WHERE` clause specifies the pattern to be matched, in this case, finding all instances of classes using the rdf schema predicate `a`. This will retrieve all instances that are classes.

--------
### Code


```sparql
PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT (COUNT(DISTINCT ?class) as ?numClasses)
WHERE {
    ?class a rdfs:Class .
}
```

--------
### Request result

ValueError: No existing entity for IRI 'http://www.w3.org/2000/01/rdf-schema#Class'! (use error_on_undefined_entities=False to accept unknown entities in SPARQL queries)

In [13]:
A = EX("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT DISTINCT ?tech ?risk ?riskSubgroup (COUNT(?riskGroup) AS ?techCount) ?techSubgroup
WHERE {
  ?techSubgroup a pbn:TechSubgroup .
  ?tech  pbn:has_TechSubgroup ?techSubgroup .
  ?risk pbn:has_RiskTechnology ?tech .
  ?risk pbn:has_RiskSubgroup ?riskSubgroup .
  FILTER (?TechSubgroup = pbn:PBN__TechSubgroup_75)  # Replace 'SpecificTechnology' with the actual technology of interest
}
GROUP BY ?TechSubgroup
ORDER BY DESC(?techCount)
LIMIT 5""")
A

[[WIP.PBN__Technology_22,
  WIP.PBN__Risk_1416,
  WIP.PBN__RiskSubgroup_3,
  7885349010,
  WIP.PBN__TechSubgroup_0]]

In [18]:
A = EX("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT DISTINCT ?tech ?risk ?riskSubgroup ?techSubgroup (COUNT(?riskSubgroup) AS ?techCount) 
WHERE {
  ?techSubgroup a pbn:TechSubgroup .
  ?techSubgroup pbn:has_SubgroupTech ?tech .
  ?tech pbn:has_TechnologyRisk ?risk .
  ?risk pbn:has_RiskSubgroup ?riskSubgroup .
  FILTER (?techSubgroup = pbn:PBN__TechSubgroup_75)  # Replace 'SpecificTechnology' with the actual technology of interest
}
GROUP BY ?riskSubgroup
ORDER BY DESC(?techCount)
LIMIT 5""")
A

[]

In [24]:
[[x[0].label[0],x[1].label[0],x[2].label[0],x[4].label[0]] for x in A]

[['Natural ventilation systems',
  'Inadequate natural ventilation in hospital and built environments',
  'Lack of emphasis on air quality',
  'Green infrastructure management'],
 ['Natural ventilation systems',
  'Inadequate natural ventilation in hospital and built environments',
  'Lack of emphasis on air quality',
  'Green infrastructure management'],
 ['Natural ventilation systems',
  'Inadequate natural ventilation in hospital and built environments',
  'Lack of emphasis on air quality',
  'Green infrastructure management'],
 ['Natural ventilation systems',
  'Inadequate natural ventilation in hospital and built environments',
  'Lack of emphasis on air quality',
  'Green infrastructure management'],
 ['Natural ventilation systems',
  'Inadequate natural ventilation in hospital and built environments',
  'Lack of emphasis on air quality',
  'Green infrastructure management']]

In [19]:
A = EX("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT DISTINCT ?tech ?risk ?riskSubgroup (COUNT(?riskGroup) AS ?techCount) ?techSubgroup
WHERE {
  ?tech a pbn:Technology .
  ?tech  pbn:has_TechSubgroup ?techSubgroup .
  ?risk pbn:has_RiskTechnology ?tech .
  ?risk pbn:has_RiskSubgroup ?riskSubgroup .
  FILTER (?tech = pbn:PBN__Technology_1010)  # Replace 'SpecificTechnology' with the actual technology of interest
}
GROUP BY ?riskGroup
ORDER BY DESC(?techCount)
LIMIT 5""")
# 26s
A

[[WIP.PBN__Technology_1010,
  WIP.PBN__Risk_4243,
  WIP.PBN__RiskSubgroup_46,
  80,
  WIP.PBN__Technology_2117]]

In [20]:
A = EX("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT DISTINCT ?tech ?risk ?riskSubgroup (COUNT(?riskGroup) AS ?techCount) ?techSubgroup
WHERE {
  ?tech a pbn:Technology .
  ?tech  pbn:has_TechSubgroup ?techSubgroup .
  ?risk pbn:has_RiskTechnology ?tech .
  ?risk pbn:has_RiskSubgroup ?riskSubgroup .
  FILTER (?tech = pbn:PBN__Technology_1010)  # Replace 'SpecificTechnology' with the actual technology of interest
}
GROUP BY ?riskGroup
ORDER BY DESC(?techCount)
LIMIT 5""")
A

[[WIP.PBN__Technology_1010,
  WIP.PBN__Risk_4243,
  WIP.PBN__RiskSubgroup_46,
  80,
  WIP.PBN__Technology_2117]]

In [28]:
import owlready2.sparql.parser
owlready2.sparql.parser._DATA_PROPS = set()

In [21]:
A = EX("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT ?tech ?risk (COUNT(?riskSubgroup) AS ?riskCount)
WHERE {
  ?riskSubgroup a pbn:RiskSubgroup .
  ?risk pbn:has_RiskSubgroup ?riskSubgroup .
  FILTER (?riskSubgroup = pbn:PBN__RiskSubgroup_46)  # Replace 'SpecificTechnology' with the actual technology of interest
}
GROUP BY ?riskSubgroup
ORDER BY DESC(?riskCount)
LIMIT 5""")
A

ValueError: Cannot select 'VAR:'?tech''!

In [ ]:
A = EX("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT ?tech ?riskSubgroup ?techSubgroup (COUNT(?riskSubgroup) AS ?riskSGCount)
WHERE {
  ?riskSubgroup a pbn:RiskSubgroup .
  ?techSubgroup a pbn:TechSubgroup .
  ?risk pbn:has_RiskSubgroup ?riskSubgroup .
  ?risk pbn:has_RiskTechnology ?tech .
  ?tech  pbn:has_TechSubgroup ?techSubgroup .
  FILTER (?riskSubgroup = pbn:PBN__RiskSubgroup_46)  # Replace 'SpecificTechnology' with the actual technology of interest
}
GROUP BY ?techSubgroup
ORDER BY DESC(?riskSGCount)
LIMIT 5""")
A

ValueError: No existing entity for IRI 'https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#TechSubgroup'! (use error_on_undefined_entities=False to accept unknown entities in SPARQL queries)

In [41]:
onto.PBN__Technology_2117.is_a

[WIP.Technology]

In [32]:
[[x[0].label[0],x[1].label[0],x[2].label[0],x[3].label[0]] for x in A]

[['Hvac and ventilation systems',
  'Natural ventilation systems',
  'Inadequate natural ventilation in hospital and built environments',
  'Lack of emphasis on air quality'],
 ['Hvac and ventilation systems',
  'Natural ventilation systems',
  'Open windows may lead to the uneven distribution of pollutants in a building, with some areas experiencing rapid reduction in pollutant concentration while others take longer to clear.',
  'Air quality and pollution effects'],
 ['Hvac and ventilation systems',
  'Natural ventilation systems',
  'Decreased transmission during summer period by 22%',
  'Climate change effects'],
 ['Hvac and ventilation systems',
  'Natural ventilation systems',
  'Human fear of infectious diseases has led to the emergence of artificial environments.',
  'Health behaviors and compliance'],
 ['Hvac and ventilation systems',
  'Natural ventilation systems',
  'High scores for lighting and electricity domains, impacting energy efficiency and potential to reduce diseas

In [ ]:
EX("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT DISTINCT ?riskGroup ?risk ?technology
WHERE {
  ?riskGroup a pbn:RiskGroup .
  ?risk pbn:has_RiskGroup ?riskGroup .
  ?risk pbn:has_RiskTechnology ?technology .
  FILTER (?riskGroup = pbn:PBN__RiskGroup_4)  # Replace 'SpecificRiskGroup' with the actual RiskGroup of interest
}
ORDER BY ?technology  # or another property if you have a ranking metric
""")

[[WIP.PBN__RiskGroup_4, WIP.PBN__Risk_2351, WIP.PBN__Technology_22],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_2132, WIP.PBN__Technology_22],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_1375, WIP.PBN__Technology_22],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_1143, WIP.PBN__Technology_22],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_2064, WIP.PBN__Technology_1575],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_2059, WIP.PBN__Technology_1568],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_2062, WIP.PBN__Technology_1573],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_47, WIP.PBN__Technology_99],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_3854, WIP.PBN__Technology_99],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_6599, WIP.PBN__Technology_277],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_10119, WIP.PBN__Technology_277],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_12628, WIP.PBN__Technology_277],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_14199, WIP.PBN__Technology_277],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_18191, WIP.PBN__Technology_277],
 [WIP.PBN__RiskGroup_

In [38]:
A = EX("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT ?riskGroup ?techGroup ?techSubgroup (COUNT(?technology) AS ?techCount)
WHERE {

  ?riskGroup a pbn:RiskGroup .
  ?techSubgroup a pbn:TechSubgroup .
  ?technology a pbn:Technology . 
  ?riskGroup a pbn:RiskGroup . 
  ?techGroup a pbn:TechGroup .

  ?risk pbn:has_RiskGroup ?riskGroup .
  ?risk pbn:has_RiskTechnology ?technology .
  ?technology pbn:has_TechSubgroup ?techSubgroup . 
  ?technology pbn:has_TechGroup ?techGroup . 
  FILTER (?riskGroup = pbn:PBN__RiskGroup_4) 
}
GROUP BY ?techSubgroup
ORDER BY DESC(?techCount)
LIMIT 5""")
A

[[WIP.PBN__RiskGroup_4, WIP.PBN__TechGroup_0, WIP.PBN__TechSubgroup_52, 459],
 [WIP.PBN__RiskGroup_4, WIP.PBN__TechGroup_0, WIP.PBN__TechSubgroup_34, 175],
 [WIP.PBN__RiskGroup_4, WIP.PBN__TechGroup_0, WIP.PBN__TechSubgroup_55, 139],
 [WIP.PBN__RiskGroup_4, WIP.PBN__TechGroup_6, WIP.PBN__TechSubgroup_47, 126],
 [WIP.PBN__RiskGroup_4, WIP.PBN__TechGroup_6, WIP.PBN__TechSubgroup_10, 117]]

In [33]:
A = EX("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT ?riskGroup ?techGroup (COUNT(?technology) AS ?techCount)
WHERE {

  ?riskGroup a pbn:RiskGroup .
  ?techGroup a pbn:TechGroup .
  ?technology a pbn:Technology . 
  ?riskGroup a pbn:RiskGroup . 

  ?risk pbn:has_RiskGroup ?riskGroup .
  ?risk pbn:has_RiskTechnology ?technology .
  ?technology pbn:has_TechGroup ?techGroup . 

  FILTER (?riskGroup = pbn:PBN__RiskGroup_4) 
}
GROUP BY ?techGroup
ORDER BY DESC(?techCount)
LIMIT 3""")
A

[[WIP.PBN__RiskGroup_4, WIP.PBN__TechGroup_0, 1020],
 [WIP.PBN__RiskGroup_4, WIP.PBN__TechGroup_6, 471],
 [WIP.PBN__RiskGroup_4, WIP.PBN__TechGroup_10, 166]]